In [1]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup

import requests

In [ ]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
driver.get(urls[0]['url'])
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")

In [2]:
url  = 'https://www.amazon.sa/s?k=Beauty+land+gardens&ref=bl_dp_s_web_0'

In [74]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
cookies = {'session': '134-8225175-0355220'}
r = requests.get(url, headers=headers, cookies=cookies)

In [ ]:
soup = BeautifulSoup(r.content, "html.parser")
products = soup.find_all('a', {'class': 'a-link-normal s-no-outline'})
len(products)
list_products = ['https://www.amazon.sa' + pro['href'] for pro in products]
list_products

In [12]:
next_page = soup.find('a', {'class': 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'})

next_page['href']

'/s?k=Beauty+land+gardens&page=2&qid=1652721454&ref=sr_pg_1'

In [2]:
url = 'https://www.amazon.sa/%D8%A3%D8%B5%D9%8A%D8%B5-%D8%B0%D8%A7%D8%AA%D9%8A-%D8%A7%D9%84%D8%B1%D9%8A%D8%8C-%D8%A8%D9%86%D9%8A%D8%8C-PTF261/dp/B0918R9QJP/ref=sr_1_3?keywords=Beauty+land+gardens&qid=1652724851&sr=8-3'
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#opti#     driver = webdriver.Firefox()ons.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
driver.get(url)
r = driver.find_element_by_tag_name('body').get_attribute('innerHTML')
soup = BeautifulSoup(r, "html.parser")

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/wafistos/anaconda3/lib/python3.8/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/wafistos/anaconda3/lib/python3.8/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17


<ipython-input-2-81824cf467a6>:8: DeprecationWarning: use options instead of firefox_options
  driver = webdriver.Firefox(firefox_options=options)


In [3]:
def return_ele(name, soup):
    try:
        return soup.find('span', text=re.compile(name)).next_element.next_element.next_element.next_element.text.strip()
    except:
        return ''
    

In [89]:
free_colors = return_ele('اللون', soup)
raw_materials = return_ele('المادة', soup)
brand = return_ele('العلامة التجارية', soup)
free_colors, raw_materials, brand

('كوفي', 'فايبر', 'بيوتي لاند جاردنز')

In [5]:
urls = pd.read_excel('amz_update_url.xlsx')
list_urls = []

for indx, row in urls.iterrows():
    list_urls.append(
        {'url': row['url'], 'cat1': row['cat1'], 'cat2': row['cat2'], 'cat3': row['cat3']},
    )

df = pd.read_excel('amz_product_model.xlsx')

for i, url1 in enumerate(list_urls[127:]):
    print('Count: ', i)
    url = url1['url']
    print('URL: ', url)
    userAgent = ua.random
    headers = {'User-Agent': userAgent}
    cookies = {'session': '134-8225175-0355220'+ str(i)}
    time.sleep(1)
    r = requests.get(url, headers=headers, cookies=cookies)
    time.sleep(1)
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        name = soup.find('span', {'id': 'productTitle'}).text.strip()
    except:
        headers = {'User-Agent': userAgent}
        cookies = {'session': '134-8225175-0355220'+ str(i)}
        time.sleep(1)
        r = requests.get(url, headers=headers, cookies=cookies)
        time.sleep(1)
        soup = BeautifulSoup(r.text, "html.parser")
        name = soup.find('span', {'id': 'productTitle'}).text.strip()

    try:
        price1 = soup.find('span', {'class': 'a-price-whole'}).text.strip().split('.')[0]
    except:
        continue
    price2 = soup.find('span', {'class': 'a-price-fraction'}).text.strip()
    price = price1 + '.'+  price2
    free_colors = return_ele('اللون', soup)
    raw_materials = return_ele('المادة', soup)
    brand = return_ele('العلامة التجارية', soup)
    try:
        product_details = soup.find('table', {'id': 'productDetails_techSpec_section_1'}).text.strip()
    except:
        product_details = ''

    is_in_stock = soup.find('div', {'id': 'availability'}).find('span').text.strip()

    description = soup.find('div', {'id': 'featurebullets_feature_div'}).text.strip()

    images = soup.find('div', {'id': 'altImages'}).find_all('img', )
    len(images)

    list_images = []
    for img in images[:-1]:
        # print(img['src'].split('_'))
        list_images.append(''.join([img['src'].split('_')[0], img['src'].split('_')[-1].replace('.', '')] ))

    base_image = list_images[0]
    add_images = ','.join(list_images[1:])
    data = {
        'name': name,
        'link_url': url,
        'price': price,
        'free_colors': free_colors,
        'raw_materials': raw_materials,
        'brand': brand,
        'product_details': product_details,
        'is_in_stock': is_in_stock,
        'description': description,
        'base_image': base_image,
        'add_images': add_images,
    }
    
    df1 = pd.DataFrame([data])
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel('amz_update_products01.xlsx')

Count:  0
URL:  https://www.amazon.sa/%D8%A8%D9%8A%D9%88%D8%AA%D9%8A-%D9%84%D8%A7%D9%86%D8%AF-%D8%AC%D8%A7%D8%B1%D8%AF%D9%86%D8%B2-%D9%86%D8%A8%D8%A7%D8%AA%D8%A7%D8%AA-%D8%B5%D9%86%D8%A7%D8%B9%D9%8A%D8%A9/dp/B09MC49C14/ref=sr_1_128?keywords=Beauty+land+gardens&qid=1652722504&sr=8-128
Count:  1
URL:  https://www.amazon.sa/%D8%AE%D8%B4%D8%A8-%D8%A7%D9%84%D8%A8%D9%82%D8%B3-%D9%85%D9%82%D8%A7%D8%B3-%D9%85%D9%88%D8%AF%D9%8A%D9%84-PL011/dp/B098XZ1ZWH/ref=sr_1_129?keywords=Beauty+land+gardens&qid=1652722504&sr=8-129
Count:  2
URL:  https://www.amazon.sa/%D8%A8%D9%8A%D9%88%D8%AA%D9%8A-%D9%84%D8%A7%D9%86%D8%AF-%D8%AC%D8%A7%D8%B1%D8%AF%D9%86%D8%B2-%D8%A7%D8%B5%D8%B7%D9%86%D8%A7%D8%B9%D9%8A%D8%8C-PL002/dp/B098YCN645/ref=sr_1_130?keywords=Beauty+land+gardens&qid=1652722504&sr=8-130
Count:  3
URL:  https://www.amazon.sa/%D8%B4%D8%AC%D8%B1%D8%A9-%D8%A7%D9%84%D8%AF%D9%8A%D9%81%D9%86%D8%A8%D8%A7%D8%AE%D9%8A%D8%A7-%D8%A7%D9%84%D8%A7%D8%B5%D8%B7%D9%86%D8%A7%D8%B9%D9%8A%D8%A9-%D8%A8%D8%B7%D9%88%D9%84-45/

In [6]:
df = None
df1 = pd.read_excel('amz_update_products0.xlsx')
df2 = pd.read_excel('amz_update_products01.xlsx')

In [7]:
df =  pd.concat([df1, df2], ignore_index=True)

In [ ]:
df.to_excel('amz_update_pro2.xlsx')